In [ ]:
import multiprocessing as mp

class Producer(mp.Process):
    def __init__(self,Buffer,BUFFER_SIZE,itemCount,lock):
        super(Producer,self).__init__()
        self.consumidor=None
        self.BUFFER_SIZE=BUFFER_SIZE
        self.Buffer=Buffer
        self.itemCount=itemCount
        self.lock=lock
        self.loop=True
        self.item=0
        
    def addConsimidor(self,consumidor):
        self.consumidor=consumidor
        
    def produceItem(self):
        self.item+=1
        print('Se produce el item:',self.item)
        return self.item

    def sleep(self):
        print("Se duerme el productor")
        self.loop=False
        
    def wakeUp(self):
        print("Se levanta el productor")
        self.loop=True
        self.run()

    def putItemIntoBuffer(self,item):
        self.lock.acquire()
        self.itemCount.value+=1
        self.Buffer.put(item)
        print("Se agrega el item {} al buffer".format(item))
        print("Numero de elementos en el buffer:",self.itemCount.value)
        self.lock.release()

    def run(self):
        while(self.loop):
            item=self.produceItem()
            if self.itemCount.value==self.BUFFER_SIZE:
                self.sleep()
          
            #elif self.itemCount.value<self.BUFFER_SIZE:
            self.putItemIntoBuffer(item)
            if self.itemCount.value == 1:
                self.consumidor.wakeUp()      

class Consumer(mp.Process):
    def __init__(self,Buffer,BUFFER_SIZE,itemCount,lock):
        super(Consumer,self).__init__()
        self.productor=None
        self.BUFFER_SIZE=BUFFER_SIZE
        self.itemCount=itemCount
        self.lock=lock
        self.Buffer=Buffer
        self.loop=True
       
    def addProductor(self,productor):
        self.productor=productor
        
    def sleep(self):
        print("Se duerme el consumidor")
        self.loop=False
        
    def wakeUp(self):
        print("Se levanta el consumidor")
        self.loop=True
        self.run()
        
    def removeItemFromBuffer(self):
        self.lock.acquire()
        getItem=self.Buffer.get()
        self.itemCount.value-=1
        print("Se remueve el item {} al buffer".format(getItem))
        print("Numero de elementos en el buffer:",self.itemCount.value)
        self.lock.release()
        return getItem

    def consumeItem(self,item):
        print('Se consume el item:',item)

    def run(self):
        while(self.loop):
            if self.itemCount.value==0:
                self.sleep()
            #elif self.itemCount.value>0:
            item=self.removeItemFromBuffer()
            self.consumeItem(item)
            if self.itemCount.value==(BUFFER_SIZE-1):
                self.productor.wakeUp()


if __name__=='__main__':
    lock=mp.Lock()
    Buffer=mp.Queue()
    BUFFER_SIZE = 4
    itemCount=mp.Value('i',0)
    productor = Producer(Buffer,BUFFER_SIZE,itemCount,lock)
    consumidor=Consumer(Buffer,BUFFER_SIZE,itemCount,lock)
    productor.addConsimidor(consumidor)
    consumidor.addProductor(productor)
    productor.start()
    consumidor.start()
    productor.join()
    consumidor.join()